In [1]:
import requests as re
import json 
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import time
from secedgar.core.cik_lookup import CIKLookup
key = json.load(open('keys.json'))['key']

ModuleNotFoundError: No module named 'secedgar'

In [2]:
# Create list of tickers to query
tickersOld = {
'BAC',
'BCS',
'C',
'COF',
'GS',
'TFC',
'V'
}



In [3]:


# Create date range to query
dateRange = {
    'rangeStartDate' : dt.datetime.strptime('2020-01-01','%Y-%m-%d'),
    'rangeEndDate' : dt.datetime.strptime('2021-04-01','%Y-%m-%d'),
} 
print(dateRange['rangeStartDate'],dateRange['rangeEndDate'])

2020-01-01 00:00:00 2021-04-01 00:00:00


In [ ]:
# Make a dataframe to store the data
tickersData = pd.DataFrame()

#quarter iteration loop
startDate = dateRange['rangeStartDate']
endDate = (startDate + relativedelta(months = 3, days = -1))
i = 1
while endDate <= dateRange['rangeEndDate']+relativedelta(months = 3, days = -1):
    
    print(i, startDate,endDate)
    # get the tickers data
    for ticker in tickers:
        while True:
            try:
                endDateExpanded = endDate + relativedelta(months = 1)
                startDateExpanded = startDate + relativedelta(months = -1)
                queryString = (f'https://api.polygon.io/vX/reference/financials?ticker={ticker}&period_of_report_date.gte={dt.datetime.strftime(startDate,"%Y-%m-%d")}&period_of_report_date.lt={dt.datetime.strftime(endDate,"%Y-%m-%d")}&timeframe=quarterly&include_sources=false&apiKey={key}')
                print(startDate, queryString)
                data = re.get(queryString)
                results = data.json().get('results')
                # get all results in a row and append to dataframe
                tickerRecord = {
                    "date": results[0]['end_date'],
                    'ticker' : ticker,
                    'revenues' : results[0]['financials']['income_statement']['revenues']['value'],
                    'net_income_loss_attributable_to_parent' : results[0]['financials']['income_statement']['net_income_loss_attributable_to_parent']['value']
                }
                tickersData= tickersData.append(tickerRecord, ignore_index=True)
                print(f'DATE RANGE: {startDate}, {endDate} \n TICKER: {ticker}, \n TICKER RECORD: {tickerRecord} \n DATA: {data}')  
                #Wait before making another request because of api rate limit
                time.sleep(15)
                break
            except:
                if (data.json().get('status') == 'error'):
                    print('error')
                    print(data.json().get('message'))
                    time.sleep(13)
                    continue
                print(data.json().get('status'),ticker, startDate,endDate, 'no data, continuing')
                tickerRecord={
                    "date": dt.datetime.strftime(startDate,"%Y-%m-%d"),
                    "ticker" : ticker,
                    "revenues" : np.NaN,
                    "net_income_loss_attributable_to_parent" :np.NaN
                }
                tickersData= tickersData.append(tickerRecord, ignore_index=True)
                time.sleep(15)
                break
    startDate = (startDate + relativedelta(months = 3,))
    startDate = startDate.replace(day=1)
    endDate =(startDate + relativedelta(months = 3, days = -1))
    i += 1



In [6]:
tickersData.to_csv('tickersData.csv')


# SEC API Attempt
https://www.sec.gov/edgar/sec-api-documentation
http://xbrl.squarespace.com/understanding-sec-xbrl-financi/

In [22]:
tickers = [
'WFC',
'LPLA',
'BAC',
'GS',
'IVZ',
'FICO',
'MTB',
'FNMA',
'FMCC',
'HBAN',
'USB',
'COF',
'TRI',
'MCI',
'GNW'
]

tickers_CIKs = CIKLookup(tickers)

In [23]:
# Dataframe setup
#Columns: Ticker | CIK | Date | Revenue | Net Income
# columns = ['ticker', 'CIK', 'filing_date','revenue', 'net_income']
tickersData = pd.DataFrame(tickers_CIKs, columns=['ticker', 'CIK'])
tickersData

,ticker,CIK,filing_date,revenue,net_income
0,WFC,NaN,NaN,NaN,NaN
1,LPLA,NaN,NaN,NaN,NaN
2,BAC,NaN,NaN,NaN,NaN
3,GS,NaN,NaN,NaN,NaN
4,IVZ,NaN,NaN,NaN,NaN
5,FICO,NaN,NaN,NaN,NaN
6,MTB,NaN,NaN,NaN,NaN
7,FNMA,NaN,NaN,NaN,NaN
8,FMCC,NaN,NaN,NaN,NaN
9,HBAN,NaN,NaN,NaN,NaN


In [14]:
queryString = (f'https://data.sec.gov/api/xbrl/companyconcept/CIK0000070858/us-gaap/ComprehensiveIncomeNetOfTax.json')
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
    'From': 'youremail@domain.com'  # This is another valid field
}
data = re.get(queryString, headers = headers)
data
results = data.json()
# results

In [15]:
results

{'cik': 70858,
 'taxonomy': 'us-gaap',
 'tag': 'ComprehensiveIncomeNetOfTax',
 'label': 'Comprehensive Income (Loss), Net of Tax, Attributable to Parent',
 'description': 'Amount after tax of increase (decrease) in equity from transactions and other events and circumstances from net income and other comprehensive income, attributable to parent entity. Excludes changes in equity resulting from investments by owners and distributions to owners.',
 'entityName': 'Bank of America Corporation',
 'units': {'USD': [{'start': '2010-01-01',
    'end': '2010-12-31',
    'val': 3315000000,
    'accn': '0000070858-13-000097',
    'fy': 2012,
    'fp': 'FY',
    'form': '10-K',
    'filed': '2013-02-28',
    'frame': 'CY2010'},
   {'start': '2011-01-01',
    'end': '2011-03-31',
    'val': 2578000000,
    'accn': '0000070858-12-000191',
    'fy': 2012,
    'fp': 'Q1',
    'form': '10-Q',
    'filed': '2012-05-03',
    'frame': 'CY2011Q1'},
   {'start': '2011-01-01',
    'end': '2011-06-30',
    'va